In [1]:
import re
import json
import os
import sys

parent_path = os.path.abspath('..')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../')
sys.path.append(parent_path)
parent_path = os.path.abspath('../../../')
sys.path.append(parent_path)

from core.ShoreNet.conf import get_data_path

DATA_PATH = get_data_path()

# load .env file
project_path = os.path.abspath('../../')
dotenv_path = os.path.join(project_path, 'secret', '.env')

from dotenv import load_dotenv
load_dotenv(dotenv_path)

print(os.getenv("SISI_DB_TYPE"))

# load project variables
from core.ShoreNet.definitions.variables import VariablesManager

var = VariablesManager()
print(var.data_path)


mysql
D:\data\sisi\


In [2]:
# load dock polygons
from core.ShoreNet.events.dock import get_dock_polygon

dock_polygon_list = get_dock_polygon(var.engine)
print(len(dock_polygon_list))

Dock polygon count: 1518
1518


# 1 Process Sail Events

## 1.1 Cleaning (DBSCAN):

1. Load whole events from SQL database
2. Clean events with DBSCAN, parameter is {radius: 200m, cluster_min_count: 30}
3. Filter noise cluster
4. Further filter: If there are less 20 ships and the count of events less than 60, the cluster will also be recognized as noise cluster.
5. After the double filter, we get the cleaned events dataframe.

## 1.2 Load Statics Data and Events Data

In [3]:
import pandas as pd

from sqlalchemy import text

from core.ShoreNet.events.filter import clean_up_events
from core.ShoreNet.statics.filter import clean_up_statics

# count all sail logging
stop_event_query = f"""
SELECT 
    mmsi, 
    Begin_time as begin_time, End_time as end_time, end_time - begin_time as duration, 
    lng, lat, Point_num as point_num, Event_categories, coal_dock_id
FROM 
    sisi.factor_stop_events t
WHERE
    t.avgSpeed < 1
"""

coal_static_query = f"""
SELECT
    mmsi, ship_name, ship_type, length, width, dwt
FROM
    sisi.dim_ships_statics
"""

events_df = pd.read_sql(
    sql=text(stop_event_query), con=var.engine
)

statics_df = pd.read_sql(
    sql=text(coal_static_query), con=var.engine
)
statics_df = clean_up_statics(statics_df)

events_df = clean_up_events(
    df=events_df,
    var=var
    # mmsi_enum_list=statics_df['mmsi'].tolist()
)

original event data shape: (1202751, 9)
cleaned event data shape: (108661, 9)


In [6]:
events_df

,mmsi,begin_time,end_time,duration,lng,lat,point_num,Event_categories,coal_dock_id,cluster
44,25,1675001337,1675008290,6953,108.332466,21.571281,6,stop_event_poly,None,0
56,31,1674046169,1674063031,16862,108.332250,21.572700,5,stop_event_poly,None,0
57,32,1675002217,1675010440,8223,108.332731,21.567901,10,stop_event_poly,None,0
63,48,1675125898,1675136459,10561,121.415280,37.593215,4,stop_event_poly,None,1
73,56,1673280014,1673306995,26981,121.443051,31.007480,7,stop_event_poly,None,-1
...,...,...,...,...,...,...,...,...,...,...
1185293,994131671,1703031100,1703405861,374761,108.348920,21.550220,132,stop_event_poly,None,2
1185294,994131671,1703435918,1703550386,114468,108.348916,21.550246,61,stop_event_poly,None,2
1185296,994131671,1703668794,1704037724,368930,108.349031,21.550266,33,stop_event_poly,None,2
1185387,999900254,1701931270,1701938449,7179,119.373750,35.088296,1,stop_event_poly,None,-1


## 1.3 DBSCAN Clustering for Events

In [4]:
from core.ShoreNet.events.dock import cluster_dock_polygon_dbscan

cleaned_evnet_df = cluster_dock_polygon_dbscan(
    events_df=events_df, var=var
)
print(f"events cluster count : {cleaned_evnet_df['cluster'].nunique()}")
with_polygon_dbscan_df = cleaned_evnet_df.loc[~cleaned_evnet_df['coal_dock_id'].isna()]
without_polygon_dbscan_df = cleaned_evnet_df.loc[cleaned_evnet_df['coal_dock_id'].isna()]
print(with_polygon_dbscan_df.shape, without_polygon_dbscan_df.shape, cleaned_evnet_df.shape)
print(f"Event with polygon percentage is {100 * with_polygon_dbscan_df.shape[0] / cleaned_evnet_df.shape[0]} %")

events cluster count : 36
(0, 10) (53529, 10) (53529, 10)
Event with polygon percentage is 0.0 %


In [5]:
cleaned_evnet_df

,mmsi,begin_time,end_time,duration,lng,lat,point_num,Event_categories,coal_dock_id,cluster
44,25,1675001337,1675008290,6953,108.332466,21.571281,6,stop_event_poly,None,0
56,31,1674046169,1674063031,16862,108.332250,21.572700,5,stop_event_poly,None,0
57,32,1675002217,1675010440,8223,108.332731,21.567901,10,stop_event_poly,None,0
85,60,1672953014,1672957079,4065,108.354500,21.555766,7,stop_event_poly,None,2
86,60,1673005432,1673008986,3554,108.329031,21.589866,29,stop_event_poly,None,0
...,...,...,...,...,...,...,...,...,...,...
1185288,994131671,1701819406,1701938972,119566,108.348900,21.550216,1,stop_event_poly,None,2
1185290,994131671,1702454390,1702528217,73827,108.348966,21.550266,94,stop_event_poly,None,2
1185293,994131671,1703031100,1703405861,374761,108.348920,21.550220,132,stop_event_poly,None,2
1185294,994131671,1703435918,1703550386,114468,108.348916,21.550246,61,stop_event_poly,None,2
